In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [910 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-08-24 04:26:16--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  3.15MB/s    in 0.3s    

2022-08-24 04:26:17 (3.15 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in amazon luggage.tsv from S3 into a DataFrame
url = "https://amzbigdatahmwrk.s3.amazonaws.com/amazon_reviews_us_Luggage_v1_00.tsv"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Luggage_v1_00.tsv"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   40884699| R9CO86UUJCAW5|B00VGTN02Y|     786681372|Teenage Mutant Ni...|         Luggage|          3|            0|          0|   N|                Y|my review of this...|my review of this...|2015-01-31 00:08:00|
|         US|   23208852|R3PR8X6QGVJ8B1|B005KIWL0E|     618251799|Kenneth Cole Reac...| 

In [5]:
print(df.count())


348657


In [6]:
customers_df = df.select("customer_id")
customers_df.show(5)

+-----------+
|customer_id|
+-----------+
|   40884699|
|   23208852|
|   17100246|
|   13533670|
|   38541771|
+-----------+
only showing top 5 rows



In [7]:
final_customer_df = customers_df.groupby("customer_id").agg({"customer_id":"count"})

final_customer_df.show(5)

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   42596797|                 1|
|   12296525|                 1|
|   49101505|                 1|
|   28377689|                 1|
|   14468845|                 1|
+-----------+------------------+
only showing top 5 rows



In [8]:
final_customer_df = final_customer_df.withColumnRenamed("count(customer_id)", "customer_count")
final_customer_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   42596797|             1|
|   12296525|             1|
|   49101505|             1|
|   28377689|             1|
|   14468845|             1|
|   26079415|             1|
|   12945150|             1|
|   42049189|             1|
|     740134|             1|
|   15123832|             1|
|   43742600|             2|
|     134205|             2|
|     255890|             1|
|   52748596|             2|
|   30070208|             1|
|   20994440|             1|
|   45330946|             1|
|   18061621|             1|
|    1051742|             1|
|   38273165|             1|
+-----------+--------------+
only showing top 20 rows



In [9]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<endpoint>:5432/amazonreviewdb"
config = {"user":"katmama",
          "password": "password",
          "driver":"org.postgresql.Driver"}

In [10]:
final_customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [12]:
products_df = df.select(["product_id", "product_title"])
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00VGTN02Y|Teenage Mutant Ni...|
|B005KIWL0E|Kenneth Cole Reac...|
|B007UNSHJ6|American Touriste...|
|B00WHFNXB4|15.5 Inch Purple ...|
|B000M5NBYU|Port & Company Zi...|
+----------+--------------------+
only showing top 5 rows



In [13]:
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [14]:
review_id = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id.show(5)

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
| R9CO86UUJCAW5|   40884699|B00VGTN02Y|     786681372|2015-01-31 00:08:00|
|R3PR8X6QGVJ8B1|   23208852|B005KIWL0E|     618251799|2015-01-31 00:08:00|
|R39BO2819ABUPF|   17100246|B007UNSHJ6|     810480328|2015-01-31 00:08:00|
|R3ADL7V6EGGEEP|   13533670|B00WHFNXB4|      20765193|2015-01-31 00:08:00|
|R1OXYPBPLVRMI5|   38541771|B000M5NBYU|     812890709|2015-01-31 00:08:00|
+--------------+-----------+----------+--------------+-------------------+
only showing top 5 rows



In [15]:
review_id.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)